# noisi_v1: Computation of ambient noise cross-correlations and noise source sensitivity kernels in python

<img src="correlation_TAM_ISP.png"  width="1200" height="20" alt="Correlation" title="" align=center />

This brief tutorial is intended to go step by step through the computation of ambient noise cross-correlations with a spatially variable distribution of noise sources. Computations are done in Python, on the basis of a database of Green's functions.

### Background
Ambient noise cross-correlations are an important seismologic observable, since they can be used to infer properties of the Earth's crust (e.g. Shapiro et al., 2005) and the ambient seismic wave field. Increasingly, cross-correlations are regarded as an observable in their own right, without equating them to empirical Green's functions. For this approach, forward modeling of cross-correlations plays an important role. Tromp et al. (2010), Hanasoge (2013) and other authors have suggested how cross-correlations can be computed without simulating the random noise field itself.

Following these authors, ambient noise cross-correlations are computed as

$$ \begin{equation}
C_{ij}(\boldsymbol{x}_1,\boldsymbol{x}_2,\tau) = \int\limits_{\partial\oplus} \left[\,G_{jm}(\boldsymbol{x}_2,\boldsymbol{\xi},t) \ast  G_{in}(\boldsymbol{x}_1,\boldsymbol{\xi},-t) \ast S_{nm}(\boldsymbol{\xi},t) \, \right] (\tau)\, d\boldsymbol{\xi}, 
\label{eq:corr_single}
\end{equation}$$

where the Green's functions $G_{ij}(\boldsymbol{x},\boldsymbol{\xi},t)$ represent wave propagation from the noise source component $ j $ at $ \xi$ to the receiver component $i $ at $x$, $ S_{nm}(\boldsymbol{\xi},t)  $ is the auto-correlation of the ambient seismic source components $n, m$ at source location $ \xi $, and the integral is over the Earth's surface Further details can be found, for example, in Tromp et al. (2010), Hanasoge (2013), Fichtner (2014).

### Using pre-computed wave fields
For simulation of noise cross-correlations (forward modeling) or pure noise source inversion (without tomographic inversion), only the parameter $S_{nm}(\boldsymbol{\xi},t)$ needs to be optimized. Hence, the Green's functions $G_{ij}(\boldsymbol{x},\boldsymbol{\xi},t)$, which usually represent the computationally most challenging part of the model, can be computed at the beginning and then recycled. This can save considerable computational effort, in particular when already existing Green's function databases such as the ones hosted on [Syngine](https://ds.iris.edu/ds/products/syngine/) (Krischer et al., 2017) are used.

Application for such an approach include ambient noise source inversion in regions where the structure is reasonably well known, or forward modelling of power spectral densities of the noise. Examples include Ermert et al. (2017) and Datta et al. (2019).

### Steps in the following tutorial (descriptions of each step will follow below):
0. Preparation
1. Setting up a new project 
2. Setting up the source grid
3. Preparing the pre-computed wave field
4. Setting up a noise source (starting) model
5. Computing cross-correlations
6. Taking measurements on the cross-correlations
7. Computing noise source sensitivity kernels


## Preparation

It is assumed that you have a working installation of [noisi_v1](https://github.com/lermert/noisi_v1).

Most calls to the noisi tool will be executed on the command line. This can be done comfortably within this jupyter notebook, prepending "!" so that the following command will be interpreted as a command line call instead of a call to python.

Two things are needed in preparation:
- execute the cell below in order to load Python modules that will be used throughout this notebook
- if you are planning on using instaseis (van Driel et al., 2015), head over to the the [syngine website](https://ds.iris.edu/ds/products/syngine/) before you start and download a Green's function database. By default, analytic Green's functions are used.

In [ ]:
# import modules which are needed later
from IPython.display import Image
import matplotlib.pyplot as plt
from noisi_v1.util.plot import plot_grid
from noisi_v1.util.add_metadata import assign_geographic_metadata
from glob import glob
from obspy import read
import numpy as np
import os

## Step 1: Setting up a new project

The noisi tool functions with a predefined directory structure, which is set up separately from the source code for each project. This is done simply by typing 

    !noisi_v1 setup_project <name_of_your_project> 

The chosen directory must not exist yet (to prevent accidental overwrites). The location can be freely chosen.

In [ ]:
# Set up your project
!noisi setup_project example

If you list the content of the project folder, the configuration file config.yml should be there:

In [ ]:
# list files there
!ls example

## Step 2: Creating a source grid
Using pre-computed wave fields is fastest by sampling them in specific source locations that are determined upfront. The source grid is therefore determined before the pre-computed wave fields are elaborated.


In this tutorial, **you can complete all steps using the default settings, or you can modify them in the configuration files** in order to construct your own example. For accessing the configuration files, move to your working directory and open them in any text editor.

The list command in the previous cell should show that the newly created directory contains two files now: A station list, which will be relevant later (stationlistc.csv) and the config file for the project (config.yml). Open this config file in an editor of your choice. You can keep the default settings or edit the parameters to your liking, in particular, the range of latitudes and longitudes (lat_min, lat_max, lon_min, lon_max), as well as the grid step (grid_dx_in_m). Then, run
  
      !noisi setup_sourcegrid <path_to_your_project> 

to set up the source grid, on which the noise source model will be defined and on which the wave field will be sampled.

In [ ]:
# create the source grid
!noisi setup_sourcegrid example
# show the sketch of the source grid
Image('example/sourcegrid.png', width=400)

This step has created a numpy binary file with the source grid, and plotted the source grid, so that we can look at it now. You should see a densely filled source grid within the latitude-longitude range of your model. Irregularities in longitude are purposefully introduced.
Note that this step only creates the spatial discretization of your modeling domain. A source distribution with geographic weights (e.g. zero sources on the continent) will be introduced in step 4.

## Step 3: Preparing the pre-computed wave field
Computation of cross-correlations is based on reusing **pre-calculated wave propagation** in between the receiver locations and the source locations as described by the grid that was defined in the previous step. This gives us the Green's functions $G_{ij}(x, \xi, t)$.

**Two ways of preparing wave fields** are available "out of the box": 

1) Using data bases from **instaseis**, which can be either computed using [AxiSEM](https://geodynamics.org/cig/software/axisem/) or downloaded from the [Syngine website](https://ds.iris.edu/ds/products/syngine/). This offers wave propagation in spherically symmetric models such as PREM, ak135 etc. 

2) Using **analytic Green's functions** for surface waves in homogeneous half-space (see Snieder 2002). 

To use one of the above out-of-the-box procedures, **edit the project configuration (config.yml)** to adapt the parameters related to the wave field. Importantly, you also need to **specify the stations** that you intend to compute cross-correlations for. Do this by editing the example station list file **(stationlist.csv)** that you find in your project folder.

Then, run

    !noisi setup_wavefield <path_to_your_project>
 

In [ ]:
# prepare the wave field
!noisi setup_wavefield --v 3000.0 --rho 3000.0 example
# for using instaseis, edit the config parameters wavefield_path and wavefield_type.
# no flags for phase velocity and density are needed when using instaseis

There should now be synthetic wave fields in the folder 
    `<path_to_your_project>/greens`
which are named by the stations in the stationlist. For illustration, we can display the example plot that was created for one station in the list. More such illustrations can be created for example by using the
        util/plot.py 
scripts, or similar.

For the sake of this tutorial, default wave field sampling is too coarse. This setting is chosen to decrease the waiting times.

If you would like to use a more sophisticated wave field from a laterally homogeneous model, you need to prepare the wave propagation run yourself and save the surface wave field. Wave propagation solvers such as Specfem, Salvus or Axisem3D already offer the option to save the surface wave field. You will need to transform the surface wave field into the format that noisi uses (an example of this format can be found in the source directory noisi_v1 under noisi_v1/test/testdata_v1/greens/)

In [ ]:
# show the sketch of the wave field:
Image('example/wavefield_example.png', width=400)


## Step 4: Creating a source starting model
The goal of this step is to create an interesting noise source model to use for the cross-correlation computation.

First, a preparatory step sets up the directory structure for the source model, similarly to how we set up a directory structure for the project. This way, one pre-computed wave field can be re-used for a range of different source models.

The directory is set up by

    !noisi setup_source <path_to_source_model>
where `<path_to_source_model>` needs to point to a location in the project directory. Similarly to earlier, this will fail it the source model directory exists.


In [ ]:
# set up the source directory
! noisi setup_source_dir example/source_1

Internally, the noise source model is parametrized as a sequence of spectra representing the different spectra of noise sources, and corresponding arrays of spatial weights for each spectrum. For example, one could define a spectrum of primary microseisms with a peak around 15 seconds and assign it high weights close to the coast, then superimpose a spectrum of secondary microseisms with a peak around 7 seconds and assign it high weights all over the ocean.

Currently, the options for defining spatial weights are: 
- homogeneous
- homogeneous in the ocean
- gaussian source blob

You can assign a weighting to each of these by **editing the source configuration file (source_setup_parameters.yml)**, also detailing other choices of the source starting model.  Re-running the same command as above sets up the starting model. (This can be a bit slow).

In [ ]:
# run a second time to initialize the source starting model
! noisi setup_source example/source_1

You should now find a source model in your source model folder under `iteration_0` by the name of `starting_model.h5`. Also, some plots of the spectra and the spatial weights should have appeared in the same directory, so let's bring them up. Spectra are plotted together, and color coded by their associated weights shown on the following maps.

In [ ]:
# show the sketch of the wave field:
display(Image('example/source_1/source_starting_model_spectra.png',
              width=400))

weight_maps = glob('example/source_1/source_starting_model_distr*.png')
weight_maps.sort()
for m in weight_maps:
    display(Image(m, width=400)) 

## Step 5: Computing cross-correlations!
Now we are ready to compute the cross-correlations themselves resulting from the source distribution determined above. Before doing so, **feel free to edit the parameters of the source configuration (source_config.yml in the source model directory)**, in particular the maximum lag of the cross-correlation (max_lag).
After that, run
    
    !noisi correlation <path_to_source_model> <iteration_nr>

specifying 0 for the iteration number (we just started).

In [ ]:
!noisi correlation example/source_1/ 0

If all went well, we have now computed a set of synthetic cross-correlations. These are saved in 
    
    <path_to_model>/<source_model>/iteration0/corr

They are saved as SAC files and can be read in, filtered, plotted etc. with [obspy](https://docs.obspy.org/).
Note that if you would like to recompute them, you need to remove them from their directory, as the tool will otherwise presume them "already computed".

In [ ]:
correlations = read('example/source_1/iteration_0/corr/*')
print(correlations)
for c in correlations[0: 2]:
    c.filter('bandpass', freqmin=0.02, freqmax=0.1, zerophase=True)
    plt.plot(np.linspace(-300, 300, c.stats.npts), c.data)
    plt.xlabel('Lag (s)')
    plt.ylabel('Cross-correlation (m^2 / s)')
    plt.xlim([-200, 200])
    plt.show()

On the cross-correlations, you may observe effects caused by the non-homogeneous source distribution that you have defined; notice that some of these effects may also stem from the finite extent of the modeling domain.

## Step 6: Taking measurements
If you plan to work on a noise source inversion with this tool, **measurements comparing your synthetic model** to observed data are the next step. For the sake of this tutorial, let us first construct synthetic data by modifying the synthetic data obtained so far:

In [ ]:
# create "synthetic data"
traces = glob('example/source_1/iteration_0/corr/*')
for file in traces:
    tr = read(file)[0]
    # change amplitudes randomly and increase the energy
    tr.data *= 100.0
    tr.write('example/source_1/observed_correlations/' + os.path.basename(file), format='SAC')
    
# this line is necessary to equip the SAC file headers with the necessary metadata
assign_geographic_metadata('example/source_1/observed_correlations', 'example/stationlist.csv')

When using other stations, you should preprocess your data and save the cross-correlations as SAC files in the observed_correlations directory of your source model. noisi expects to find the information it requires for measurements in the SAC header of the files; a list of headers is provided in the project directory (ata_sac_headers.txt).

Currently, **3 measurements are implemented**: Windowed signal energy (energy_diff), logarithmic signal energy ratio (ln_energy_ratio), and windowed waveform (windowed_waveform). To choose one of these, **edit the measurement configuration file (measr_config.yml)** in the source model directory. The windowed measurements also require setting parameters for the windows, in particular, their halfwidth and the expected Rayleigh wave group velocity (so that measurements can be taken on the Rayleigh wave). Optionally add a bandpass filter. These measurements are also described and used by Sager et al. (2018).

You can then run the measurements by calling

    !noisi measurement <path_to_source_model> <iteration_nr>
 

If using windowed measurements, you may receive the message "Windows overlap, skipping": This occurs when the causal and acausal windows overlap and the win_overlap parameter is set to false.

In [ ]:
# Take the measurement
! noisi measurement example/source_1 0

The output file contains a range of information for each measurement, namely the station identifiers, their geographic position, the distance between them, measurement values, l2 misfit, signal to noise ratios, and number of stacked windows of the observed data.

In [ ]:
# check the output file
! head example/source_1/iteration_0/energy_diff.0.measurement.csv  
# or substitute name of other measurement that was used

## Step 7: Computing source sensitivity
In a final step, we can now **evaluate the sensitivity of the above measurement to changes in the source distribution** (here, only to the overall geographic source distribution). This set of sensitivity kernels can be used for (iterative or linearised) inversion. It is computed by calling 

    ! noisi_v1 kernel <path_to_source_model> <iteration_nr>

In [ ]:
# compute kernels
! noisi kernel example/source_1 0

Once this is completed, the kernels can be plotted. They are saved in the `<path_to_source_model>/iteration_0` directory. Each numpy binary file corresponds to one bandpass filter (numbered in the order of filters specified the measr_config.yml file) and one station pair. The files contain a single array of sensitivity values ordered corresponding to the source grid in `<path_to_project>/sourcegrid.npy`. The `plot_grid` function provides a simple way to plot them for a first look.

In [ ]:
# plot the kernel
kern = np.load('example/source_1/iteration_0/kern/G.SSB..MXZ--MN.BNI..MXZ.0.npy')
grd = np.load('example/sourcegrid.npy')
plot_grid(grd[0], grd[1], -kern[0, :, 0], normalize=True, v=0.2, stations=[[45.052, 6.678], [45.279, 4.542]])
plot_grid(grd[0], grd[1], -kern[1, :, 1], normalize=True, v=0.2, stations=[[45.052, 6.678], [45.279, 4.542]])
plot_grid(grd[0], grd[1], -kern[2, :, 0], normalize=True, v=0.2, stations=[[45.052, 6.678], [45.279, 4.542]])

We have plotted the *negative* kernel here, which shows the update direction to decrease source misfit. Since in creating synthetic data, we have merely increased the energy compared to the forward model, the kernel shows that an *increase* in source PSD will lead to a more fitting model.

## Further steps...
On the basis of the above steps, one can run forward modeling and 'noise' source inversion. 

The tool computes the large number of cross-correlations and kernels as embarrassingly parallel task. To make use of this parallelization, the only required change is to call the noisi commands with mpirun, such as

    mpirun -np 3 noisi <command> options...
   
This feature is best used on a workstation or cluster.

As an example, you can re-compute the correlations below (first you have to remove them from the `corr` directory).

In [ ]:
# Compute the correlations again, this time in parallel
! rm example/source_1/iteration_0/corr/*
! mpirun -np 3 noisi correlation example/source_1 0


## References

- Tromp, J., Luo, Y., Hanasoge, S., & Peter, D. (2010). Noise cross-correlation sensitivity kernels. Geophys. J. Int., 183 , 791-819.

- Hanasoge, S. M. (2013a), Measurements and kernels for source-structure inversions in noise tomography, Geophys. J. Int., 196(2), 971–985.

- Fichtner, A. (2014). Source and processing effects on noise correlations. Geophys. J. Int., 197 , 1527-1531.

- Krischer, L., Hutko, A., van Driel, M., Stähler, S., Bahavar, M., Trabant, C., and Nissen‐Meyer, T. (2017). On‐demand custom broadband synthetic seismograms, Seismol. Res. Lett. 88, no. 4.

- Ermert, L., Sager, K., Afanasiev, M., Boehm, C., & Fichtner, A. (2017). Ambient seismic source inversion in a heterogeneous Earth - Theory and application to the Earth’s hum. J. Geophys. Res.

- Datta, A., Hanasoge, S., & Goudswaard, J. (2019). Finite frequency inversion of cross-correlation amplitudes for ambient noise source directivity estimation. EarthArXiv Preprint.

- van Driel, M., Krischer, L., Stähler, S. C., Hosseini, K., & Nissen-Meyer, T. (2015). Instaseis: instant global seismograms based on a broadband waveform database. Solid Earth, 6 (2), 701–717.

- Snieder, R. (2002). Scattering of Surface Waves. In: Scattering and Inverse Scattering in Pure and Applied Science. Elsevier.

- Sager, K., Ermert, L., Boehm, C., & Fichtner, A. Towards full waveform ambient noise inversion (2018). Geophys. J. Int., 212 (1), 566-590.


